In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import statsmodels.api as sm
import shap
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from  sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

In [31]:
import os
working_directory = os.getcwd()
print(working_directory)

/Users/ilinamitra


In [32]:
path = working_directory + '/Documents/df_updated.csv'
df = pd.read_csv(path, encoding='latin-1')
df.head()

,County,County.Code,suicides,Population,Crude_Rate,Uninsured: %,Mental_Health_Providers,Social_Associations,vet_pop_2010,Opioid Dispensing Rate per 100,...,in_labor_force,civilian_labot_force,employed,unemployed,armed_force,not_labor_force,total_occupied_housing,owner_occupied,renter_occupied,median_gross_rent
0,"Baldwin County, AL",1003,51,229287,22.242866,10.9,228,226,19614,65.0,...,99495,99317,95091,4226,178,72802,80930,60896,20034,1020
1,"Jefferson County, AL",1073,83,655342,12.665143,10.0,1379,953,42503,139.0,...,326672,326101,304994,21107,571,198562,261231,163799,97432,900
2,"Lee County, AL",1081,21,166831,12.587589,10.5,178,144,11532,35.4,...,79457,78849,74770,4079,608,50796,60237,37111,23126,856
3,"Madison County, AL",1089,54,379453,14.231011,9.9,613,403,33657,87.3,...,187581,186248,176910,9338,1333,104491,148189,99046,49143,857
4,"Mobile County, AL",1097,55,412716,13.326355,12.0,441,463,30347,98.0,...,186097,185327,174519,10808,770,141286,156251,100853,55398,853


In [33]:
df.shape #here we limited to counties bigger than or equal to population of 150,000 with 29 counties missing

(447, 113)

In [34]:
df = df.drop(["County", "County.Code","Year", "suicides","StateAbbr", "StateDesc", "LocationName", "State", "...1", 
        "total_median_age", "female_median_age","total_population", "black_pop", "asian_pop", 
         "islander_pop", "other_pop", "mixed_pop", "family_household", "other_family", "single_man_family", 
         "single_woman_family", "nonfamily_household", "single_family_alone", "single_family_notalone",
         "nursery", "kindergarten", "grade_1", "grade_2", "grade_3", "grade_4", "grade_5", "grade_6", "grade_7",
         "grade_8", "grade_9", "grade_10", "grade_11", "grade_12_no_diploma", "hs_diploma", "GED", "some_college",
         "some_college_no_degree", "associates_degree", "bachelors_degree", "professional_degree", "doctoral_degree",
        "under_0.5_ip", "0.5_0.9_ip", "1_1.24_ip", "1.25_1.49_ip", "1.5_1.84_ip", "1.85_1.99_ip",
        "assistance_income", "no_assistance_income", "income_per_capita", "total_emp", "in_labor_force", 
         "civilian_labot_force", "employed", "armed_force", "not_labor_force", "owner_occupied", "renter_occupied"],axis=1)

In [35]:
df.corr()

,Population,Crude_Rate,Uninsured: %,Mental_Health_Providers,Social_Associations,vet_pop_2010,Opioid Dispensing Rate per 100,homicide_rate,firearm_death_rate,RUCC_2013,...,native_pop,married_couple_family,no_school,masters_degree,over_2_ip,median_household_income,total_public_assistance,unemployed,total_occupied_housing,median_gross_rent
Population,1.000000,-0.252804,0.063481,0.877764,0.934661,0.882352,-0.160491,0.080147,0.077299,-0.318043,...,0.725598,0.991308,0.933810,0.917353,0.992931,0.095788,0.994499,0.978537,0.994499,0.301122
Crude_Rate,-0.252804,1.000000,0.194500,-0.231664,-0.285082,-0.102036,0.295434,-0.080498,-0.052294,0.353708,...,0.019833,-0.251948,-0.233401,-0.328589,-0.270581,-0.338723,-0.250976,-0.253076,-0.250976,-0.364547
Uninsured: %,0.063481,0.194500,1.000000,-0.128070,-0.025777,0.115507,0.135101,0.181262,0.201484,0.127092,...,0.104505,0.029473,0.086540,-0.084604,0.007111,-0.442580,0.036935,0.055326,0.036935,-0.234216
Mental_Health_Providers,0.877764,-0.231664,-0.128070,1.000000,0.894341,0.726243,-0.105732,0.065685,0.062458,-0.271186,...,0.628131,0.870082,0.854678,0.886246,0.890545,0.155454,0.889982,0.872321,0.889982,0.356331
Social_Associations,0.934661,-0.285082,-0.025777,0.894341,1.000000,0.823149,-0.058133,0.157750,0.152470,-0.336607,...,0.623654,0.931573,0.835056,0.937803,0.941095,0.115428,0.954691,0.913253,0.954691,0.282021
vet_pop_2010,0.882352,-0.102036,0.115507,0.726243,0.823149,1.000000,-0.077981,0.115424,0.117665,-0.303464,...,0.732364,0.884844,0.724963,0.784772,0.874115,0.029384,0.878648,0.841771,0.878648,0.238296
Opioid Dispensing Rate per 100,-0.160491,0.295434,0.135101,-0.105732,-0.058133,-0.077981,1.000000,0.450495,0.469528,0.247195,...,-0.079865,-0.183153,-0.169100,-0.193618,-0.185252,-0.503118,-0.136409,-0.132504,-0.136409,-0.473870
homicide_rate,0.080147,-0.080498,0.181262,0.065685,0.157750,0.115424,0.450495,1.000000,0.998613,-0.076464,...,0.031290,0.023105,0.056824,0.033559,0.042149,-0.417371,0.101530,0.151772,0.101530,-0.217514
firearm_death_rate,0.077299,-0.052294,0.201484,0.062458,0.152470,0.117665,0.469528,0.998613,1.000000,-0.068737,...,0.040624,0.020195,0.054482,0.024980,0.038045,-0.437825,0.097792,0.148901,0.097792,-0.238442
RUCC_2013,-0.318043,0.353708,0.127092,-0.271186,-0.336607,-0.303464,0.247195,-0.076464,-0.068737,1.000000,...,-0.125520,-0.336528,-0.213850,-0.372456,-0.344745,-0.431209,-0.325859,-0.282141,-0.325859,-0.391138


# Imputation

In [39]:
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)

imputer = imputer.fit(df[['homicide_rate']])
df['homicide_rate'] = imputer.transform(df[['homicide_rate']])

imputer = imputer.fit(df[['firearm_death_rate']])
df['firearm_death_rate'] = imputer.transform(df[['firearm_death_rate']])

imputer = imputer.fit(df[['Stroke among adults aged >=18 years']])
df['Stroke among adults aged >=18 years'] = imputer.transform(df[['Stroke among adults aged >=18 years']])

imputer = imputer.fit(df[['Current lack of health insurance among adults aged 18-64 years']])
df['Current lack of health insurance among adults aged 18-64 years'] = imputer.transform(
    df[['Current lack of health insurance among adults aged 18-64 years']])

imputer = imputer.fit(df[['Obesity among adults aged >=18 years']])
df['Obesity among adults aged >=18 years'] = imputer.transform(
    df[['Obesity among adults aged >=18 years']])

imputer = imputer.fit(df[['Current asthma among adults aged >=18 years']])
df['Current asthma among adults aged >=18 years'] = imputer.transform(df[['Current asthma among adults aged >=18 years']])

imputer = imputer.fit(df[['Diagnosed diabetes among adults aged >=18 years']])
df['Diagnosed diabetes among adults aged >=18 years'] = imputer.transform(
    df[['Diagnosed diabetes among adults aged >=18 years']])

imputer = imputer.fit(df[['Depression among adults aged >=18 years']])
df['Depression among adults aged >=18 years'] = imputer.transform(
    df[['Depression among adults aged >=18 years']])

imputer = imputer.fit(df[['Arthritis among adults aged >=18 years']])
df['Arthritis among adults aged >=18 years'] = imputer.transform(
    df[['Arthritis among adults aged >=18 years']])

imputer = imputer.fit(df[['Chronic obstructive pulmonary disease among adults aged >=18 years']])
df['Chronic obstructive pulmonary disease among adults aged >=18 years'] = imputer.transform(
    df[['Chronic obstructive pulmonary disease among adults aged >=18 years']])

imputer = imputer.fit(df[['Binge drinking among adults aged >=18 years']])
df['Binge drinking among adults aged >=18 years'] = imputer.transform(
    df[['Binge drinking among adults aged >=18 years']])

imputer = imputer.fit(df[['Current smoking among adults aged >=18 years']])
df['Current smoking among adults aged >=18 years'] = imputer.transform(
    df[['Current smoking among adults aged >=18 years']])

imputer = imputer.fit(df[['Physical health not good for >=14 days among adults aged >=18 years']])
df['Physical health not good for >=14 days among adults aged >=18 years'] = imputer.transform(
    df[['Physical health not good for >=14 days among adults aged >=18 years']])

imputer = imputer.fit(df[['Coronary heart disease among adults aged >=18 years']])
df['Coronary heart disease among adults aged >=18 years'] = imputer.transform(
    df[['Coronary heart disease among adults aged >=18 years']])

imputer = imputer.fit(df[['No leisure-time physical activity among adults aged >=18 years']])
df['No leisure-time physical activity among adults aged >=18 years'] = imputer.transform(
    df[['No leisure-time physical activity among adults aged >=18 years']])

imputer = imputer.fit(df[['High blood pressure among adults aged >=18 years']])
df['High blood pressure among adults aged >=18 years'] = imputer.transform(
    df[['High blood pressure among adults aged >=18 years']])

imputer = imputer.fit(df[['Mental health not good for >=14 days among adults aged >=18 years']])
df['Mental health not good for >=14 days among adults aged >=18 years'] = imputer.transform(
    df[['Mental health not good for >=14 days among adults aged >=18 years']])

imputer = imputer.fit(df[['Mental health not good for >=14 days among adults aged >=18 years']])
df['Mental health not good for >=14 days among adults aged >=18 years'] = imputer.transform(
    df[['Mental health not good for >=14 days among adults aged >=18 years']])

imputer = imputer.fit(df[['Fair or poor self-rated health status among adults aged >=18 years']])
df['Fair or poor self-rated health status among adults aged >=18 years'] = imputer.transform(
    df[['Fair or poor self-rated health status among adults aged >=18 years']])

imputer = imputer.fit(df[['Cholesterol screening among adults aged >=18 years']])
df['Cholesterol screening among adults aged >=18 years'] = imputer.transform(
    df[['Cholesterol screening among adults aged >=18 years']])

imputer = imputer.fit(df[['Chronic kidney disease among adults aged >=18 years']])
df['Chronic kidney disease among adults aged >=18 years'] = imputer.transform(
    df[['Chronic kidney disease among adults aged >=18 years']])

imputer = imputer.fit(df[['Visits to doctor for routine checkup within the past year among adults aged >=18 years']])
df['Visits to doctor for routine checkup within the past year among adults aged >=18 years'] = imputer.transform(
    df[['Visits to doctor for routine checkup within the past year among adults aged >=18 years']])

imputer = imputer.fit(df[['Cancer (excluding skin cancer) among adults aged >=18 years']])
df['Cancer (excluding skin cancer) among adults aged >=18 years'] = imputer.transform(
    df[['Cancer (excluding skin cancer) among adults aged >=18 years']])

imputer = imputer.fit(df[['Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure']])
df['Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure'] = imputer.transform(
    df[['Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure']])

imputer = imputer.fit(df[['High cholesterol among adults aged >=18 years who have been screened in the past 5 years']])
df['High cholesterol among adults aged >=18 years who have been screened in the past 5 years'] = imputer.transform(
    df[['High cholesterol among adults aged >=18 years who have been screened in the past 5 years']])


In [41]:
y = df['Crude_Rate'] #setting y variable 
X = df.loc[:, df.columns != 'Crude_Rate'] #setting X's

In [42]:
#ML models only work if X and Y are numbers; some models also require each feature to be standardized
for i in range(10):
    random_state = 42 * i 
    
    # split the data
    X_train, X_other, y_train, y_other = train_test_split(X, y, train_size=0.8, random_state=random_state)
    X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, train_size=0.2, random_state=random_state)
    
    
    #StandardScaler standardizes continuous features by removing the mean and scaling to unit variance
    std_ftrs = ["Population", "Uninsured: %", "Mental_Health_Providers", "Social_Associations",
                "vet_pop_2010", "Opioid Dispensing Rate per 100", "homicide_rate", "avg_temp", "firearm_death_rate", 
                "precip", "Current lack of health insurance among adults aged 18-64 years", 
                "Obesity among adults aged >=18 years", "Stroke among adults aged >=18 years", 
                "Current asthma among adults aged >=18 years", "Diagnosed diabetes among adults aged >=18 years",
                "Depression among adults aged >=18 years", "Arthritis among adults aged >=18 years", 
                "Chronic obstructive pulmonary disease among adults aged >=18 years", 
                "Binge drinking among adults aged >=18 years", "Current smoking among adults aged >=18 years",
                "Physical health not good for >=14 days among adults aged >=18 years",
                "Coronary heart disease among adults aged >=18 years", 
                "No leisure-time physical activity among adults aged >=18 years",
                "High blood pressure among adults aged >=18 years", 
                "Mental health not good for >=14 days among adults aged >=18 years",
                "Fair or poor self-rated health status among adults aged >=18 years",
                "Cholesterol screening among adults aged >=18 years",
                "Chronic kidney disease among adults aged >=18 years",
                "Visits to doctor for routine checkup within the past year among adults aged >=18 years",
                "Cancer (excluding skin cancer) among adults aged >=18 years",
                "Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure",
                "High cholesterol among adults aged >=18 years who have been screened in the past 5 years",
                "dem_perc", "repub_perc", "Score", "Gun_Death_Rate_100k", "male_median_age", "white_pop", "native_pop",
                "married_couple_family", "no_school", "masters_degree", "over_2_ip", "median_household_income",
                "total_public_assistance", "unemployed", "total_occupied_housing", "median_gross_rent"]
    
    std_scaler = StandardScaler()
    std_fit = std_scaler.fit(X_train[std_ftrs])
    std_train = std_scaler.transform(X_train[std_ftrs])
    std_val = std_scaler.transform(X_val[std_ftrs])
    std_test = std_scaler.transform(X_test[std_ftrs]) 
    
    
    #encoding for categorical variables that cannot be ordered; converts categorical features into dummy arrays
    onehot_ftrs = ['RUCC_2013']
    enc = OneHotEncoder(sparse=False,handle_unknown='ignore')  #initialize encoder
    enc.fit(X_train[onehot_ftrs])  #fit the training data
    onehot_train = enc.transform(X_train[onehot_ftrs]) #transform the training data
    onehot_val = enc.transform(X_val[onehot_ftrs]) #transform validation data
    onehot_test = enc.transform(X_test[onehot_ftrs]) #transform test data
    
    X_train_concat = np.concatenate([onehot_train, std_train], axis=1)
    X_val_concat = np.concatenate([onehot_val, std_val], axis=1)
    X_test_concat = np.concatenate([onehot_test, std_test], axis=1)
    
    X_pre_col = list(enc.get_feature_names_out(['RUCC_2013'])) + std_ftrs
    X_train = pd.DataFrame(X_train_concat, columns=X_pre_col)
    X_val = pd.DataFrame(X_val_concat, columns=X_pre_col)
    X_test = pd.DataFrame(X_test_concat, columns=X_pre_col)

In [40]:
df.isnull().sum()

Population                                                                                               0
Crude_Rate                                                                                               0
Uninsured: %                                                                                             0
Mental_Health_Providers                                                                                  0
Social_Associations                                                                                      0
vet_pop_2010                                                                                             0
Opioid Dispensing Rate per 100                                                                           0
homicide_rate                                                                                            0
firearm_death_rate                                                                                       0
RUCC_2013                            

# Lasso Regression

In [44]:
y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (80/20)
    The R2 is maximized in cross-validation.
    '''
    
    test_scores = []
    best_models = []
    
    # loop through 10 random states 
    for i in range(0,10):
        random_state = 42*i
    
        # split the data
        X_train, X_other, y_train, y_other = train_test_split(X, y, train_size=0.8, random_state=random_state)
        X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, train_size=0.2, random_state=random_state)
    
    
        std_ftrs = ["Population", "Uninsured: %", "Mental_Health_Providers", "Social_Associations",
                "vet_pop_2010", "Opioid Dispensing Rate per 100", "homicide_rate", "avg_temp", 'firearm_death_rate',
                "precip", "Current lack of health insurance among adults aged 18-64 years", 
                "Obesity among adults aged >=18 years", "Stroke among adults aged >=18 years", 
                "Current asthma among adults aged >=18 years", "Diagnosed diabetes among adults aged >=18 years",
                "Depression among adults aged >=18 years", "Arthritis among adults aged >=18 years", 
                "Chronic obstructive pulmonary disease among adults aged >=18 years", 
                "Binge drinking among adults aged >=18 years", "Current smoking among adults aged >=18 years",
                "Physical health not good for >=14 days among adults aged >=18 years",
                "Coronary heart disease among adults aged >=18 years", 
                "No leisure-time physical activity among adults aged >=18 years",
                "High blood pressure among adults aged >=18 years", 
                "Mental health not good for >=14 days among adults aged >=18 years",
                "Fair or poor self-rated health status among adults aged >=18 years",
                "Cholesterol screening among adults aged >=18 years",
                "Chronic kidney disease among adults aged >=18 years",
                "Visits to doctor for routine checkup within the past year among adults aged >=18 years",
                "Cancer (excluding skin cancer) among adults aged >=18 years",
                "Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure",
                "High cholesterol among adults aged >=18 years who have been screened in the past 5 years",
                "dem_perc", "repub_perc", "Score", "Gun_Death_Rate_100k", "male_median_age", "white_pop", "native_pop",
                "married_couple_family", "no_school", "masters_degree", "over_2_ip", "median_household_income",
                "total_public_assistance", "unemployed", "total_occupied_housing", "median_gross_rent"]
    
    
        std_scaler = StandardScaler()
        std_fit = std_scaler.fit(X_train[std_ftrs])
        std_train = std_scaler.transform(X_train[std_ftrs])
        std_val = std_scaler.transform(X_val[std_ftrs])
        std_test = std_scaler.transform(X_test[std_ftrs]) 

        onehot_ftrs = ['RUCC_2013']
        enc = OneHotEncoder(sparse=False,handle_unknown='ignore')  #initialize encoder
        enc.fit(X_train[onehot_ftrs])  #fit the training data
        onehot_train = enc.transform(X_train[onehot_ftrs]) #transform the training data
        onehot_val = enc.transform(X_val[onehot_ftrs]) #transform validation data
        onehot_test = enc.transform(X_test[onehot_ftrs]) #transform test data

        X_train_concat = np.concatenate([onehot_train, std_train], axis=1)
        X_val_concat = np.concatenate([onehot_val, std_val], axis=1)
        X_test_concat = np.concatenate([onehot_test, std_test], axis=1)

        X_pre_col = list(enc.get_feature_names_out(['RUCC_2013'])) + std_ftrs
        X_train = pd.DataFrame(X_train_concat, columns=X_pre_col)
        X_val = pd.DataFrame(X_val_concat, columns=X_pre_col)
        X_test = pd.DataFrame(X_test_concat, columns=X_pre_col)
        
        #kfold split
        kf = KFold(n_splits=4,shuffle=True,random_state=42*i)
        
        # loop through the hyperparameter combinations or use GridSearchCV 
        grid = GridSearchCV(estimator = ML_algo, param_grid=param_grid,scoring = 'r2', 
                            n_jobs = -1, cv = kf, return_train_score = True, verbose=True) 
        
        
        grid.fit(X_train, y_train)
        results = pd.DataFrame(grid.cv_results_)
       

        print('best model parameters:',grid.best_params_)
        print('validation score:',grid.best_score_) # this is the mean validation score over all iterations
        # save the model
        best_models.append(grid)
        # calculate and save the test score
        y_test_pred = best_models[-1].predict(X_test)
        test_scores.append(r2_score(y_test,y_test_pred))
        print('test score:',test_scores[i])
        
        
    print('test_mean:', np.mean(test_scores))   
    print('test_std:', np.std(test_scores))
    return best_models, test_scores

ML_algo = Lasso()
param_grid = {'alpha':[0.1, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2]}

models, scores = MLpipe(X,y,ML_algo,param_grid)
print(scores)  

Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: 0.5367468876207527
test score: 0.5236500429444901
Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: 0.5077168568168089
test score: 0.6216611136308632
Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: 0.522119604341298
test score: 0.5595707074672223
Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: 0.5674267972416807
test score: 0.5522546604614614
Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: 0.5183124593457665
test score: 0.6423613274084157
Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: 0.5098569305768785
test score: 0.5601983592596621

In [45]:
y = df['Crude_Rate']
X = df.loc[:, df.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (80/20)
    The R2 is maximized in cross-validation.
    '''
    
    test_scores = []
    best_models = []
    
    # loop through 10 random states 
    for i in range(0,10):
        random_state = 42*i
    
        # split the data
        X_train, X_other, y_train, y_other = train_test_split(X, y, train_size=0.8, random_state=random_state)
        X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, train_size=0.2, random_state=random_state)
    
    
        std_ftrs = ["Population", "Uninsured: %", "Mental_Health_Providers", "Social_Associations",
                "vet_pop_2010", "Opioid Dispensing Rate per 100", "homicide_rate", "avg_temp", 'firearm_death_rate',
                "precip", "Current lack of health insurance among adults aged 18-64 years", 
                "Obesity among adults aged >=18 years", "Stroke among adults aged >=18 years", 
                "Current asthma among adults aged >=18 years", "Diagnosed diabetes among adults aged >=18 years",
                "Depression among adults aged >=18 years", "Arthritis among adults aged >=18 years", 
                "Chronic obstructive pulmonary disease among adults aged >=18 years", 
                "Binge drinking among adults aged >=18 years", "Current smoking among adults aged >=18 years",
                "Physical health not good for >=14 days among adults aged >=18 years",
                "Coronary heart disease among adults aged >=18 years", 
                "No leisure-time physical activity among adults aged >=18 years",
                "High blood pressure among adults aged >=18 years", 
                "Mental health not good for >=14 days among adults aged >=18 years",
                "Fair or poor self-rated health status among adults aged >=18 years",
                "Cholesterol screening among adults aged >=18 years",
                "Chronic kidney disease among adults aged >=18 years",
                "Visits to doctor for routine checkup within the past year among adults aged >=18 years",
                "Cancer (excluding skin cancer) among adults aged >=18 years",
                "Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure",
                "High cholesterol among adults aged >=18 years who have been screened in the past 5 years",
                "dem_perc", "repub_perc", "Score", "Gun_Death_Rate_100k", "male_median_age", "white_pop", "native_pop",
                "married_couple_family", "no_school", "masters_degree", "over_2_ip", "median_household_income",
                "total_public_assistance", "unemployed", "total_occupied_housing", "median_gross_rent"]
    
    
        std_scaler = StandardScaler()
        std_fit = std_scaler.fit(X_train[std_ftrs])
        std_train = std_scaler.transform(X_train[std_ftrs])
        std_val = std_scaler.transform(X_val[std_ftrs])
        std_test = std_scaler.transform(X_test[std_ftrs]) 

        onehot_ftrs = ['RUCC_2013']
        enc = OneHotEncoder(sparse=False,handle_unknown='ignore')  #initialize encoder
        enc.fit(X_train[onehot_ftrs])  #fit the training data
        onehot_train = enc.transform(X_train[onehot_ftrs]) #transform the training data
        onehot_val = enc.transform(X_val[onehot_ftrs]) #transform validation data
        onehot_test = enc.transform(X_test[onehot_ftrs]) #transform test data

        X_train_concat = np.concatenate([onehot_train, std_train], axis=1)
        X_val_concat = np.concatenate([onehot_val, std_val], axis=1)
        X_test_concat = np.concatenate([onehot_test, std_test], axis=1)

        X_pre_col = list(enc.get_feature_names_out(['RUCC_2013'])) + std_ftrs
        X_train = pd.DataFrame(X_train_concat, columns=X_pre_col)
        X_val = pd.DataFrame(X_val_concat, columns=X_pre_col)
        X_test = pd.DataFrame(X_test_concat, columns=X_pre_col)
        
        #kfold split
        kf = KFold(n_splits=4,shuffle=True,random_state=42*i)
        
        # loop through the hyperparameter combinations or use GridSearchCV 
        grid = GridSearchCV(estimator = ML_algo, param_grid=param_grid,scoring = 'neg_mean_absolute_error', 
                            n_jobs = -1, cv = kf, return_train_score = True, verbose=True) 
        
        
        grid.fit(X_train, y_train)
        results = pd.DataFrame(grid.cv_results_)
       

        print('best model parameters:',grid.best_params_)
        print('validation score:',grid.best_score_) # this is the mean validation score over all iterations
        # save the model
        best_models.append(grid)
        # calculate and save the test score
        y_test_pred = best_models[-1].predict(X_test)
        test_scores.append(mean_absolute_error(y_test,y_test_pred))
        print('test score:',test_scores[i])
        
        
    print('test_mean:', np.mean(test_scores))   
    print('test_std:', np.std(test_scores))
    return best_models, test_scores

ML_algo = Lasso()
param_grid = {'alpha':[0.1, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2]}

models, scores = MLpipe(X,y,ML_algo,param_grid)
print(scores) 

Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: -2.3761138457396522
test score: 3.00207491615688
Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: -2.576007410215099
test score: 2.3613823420443043
Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: -2.471789346809018
test score: 3.0193336451724764
Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: -2.549463324867174
test score: 2.1777437260654953
Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: -2.6169285818035792
test score: 2.1992291719011288
Fitting 4 folds for each of 10 candidates, totalling 40 fits
best model parameters: {'alpha': 0.1}
validation score: -2.566592822988872
test score: 2.651199243521475

In [46]:
lasso = models[3]

In [47]:
#zeros are dropped
lasso.best_estimator_.coef_

array([-0.        , -0.        ,  0.        , -0.        ,  0.        ,
       -0.        ,  0.35556801,  0.        , -0.24991002,  0.49065956,
        0.65168777, -0.1983895 ,  0.        , -0.        , -0.3772672 ,
        0.        , -0.57986912,  0.        ,  0.18387923, -0.        ,
        0.21051444,  0.51677909,  0.        , -0.26889663,  0.43632882,
        0.        ,  0.        , -0.        ,  0.        , -0.        ,
       -1.02066493, -0.49297029, -0.        , -0.92325197,  0.        ,
       -0.89330617,  0.        , -1.00643569,  0.        , -0.        ,
        1.11823262,  2.01096313, -0.77468797,  0.29215391, -0.        ,
       -0.        , -0.        , -0.        , -1.27659572, -0.        ,
       -0.        , -0.        ,  0.        ])

In [48]:
pd.Series(lasso.best_estimator_.coef_, index = X_train.columns)

RUCC_2013_1                                                                                             -0.000000
RUCC_2013_2                                                                                             -0.000000
RUCC_2013_3                                                                                              0.000000
RUCC_2013_4                                                                                             -0.000000
RUCC_2013_5                                                                                              0.000000
Population                                                                                              -0.000000
Uninsured: %                                                                                             0.355568
Mental_Health_Providers                                                                                  0.000000
Social_Associations                                                                     

In [111]:
lasso_best = Lasso(0.1)
lasso_best.fit(X_train, y_train)

Lasso(alpha=0.1)